# Simple White Island Crater Floor Map With Leveling Sites

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import requests

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM

from osgeo import gdal, osr
gdal.UseExceptions()

%matplotlib inline

In [ ]:
df = pd.read_csv('leveling_sites_current.csv')
df.head()

In [ ]:
df['ID'] = df['siteID'].apply(lambda x: x[2:])

In [ ]:
#imagery
fname = 'WhiteIsland.tif'
ds = gdal.Open(fname)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
proj = ds.GetProjection()
inproj = osr.SpatialReference()
inproj.ImportFromWkt(proj)
#projcs = inproj.GetAuthorityCode('PROJCS') does not return projection as final line missing that is shown
#in https://ocefpaf.github.io/python4oceanographers/blog/2015/03/02/geotiff/
#hard code epsg code
projection = ccrs.epsg('2193') #for NZTM
#projection = ccrs.PlateCarree() #for WGS84

#plot map
subplot_kw = dict(projection=projection)
fig,ax = plt.subplots(subplot_kw=subplot_kw, dpi=200)
extent = (gt[0], gt[0] + ds.RasterXSize * gt[1], gt[3] + ds.RasterYSize * gt[5], gt[3])
img = ax.imshow(data[:3, :, :].transpose((1, 2, 0)), extent=extent, origin='upper')

#observations as coloured points
plt.scatter(df['lon'], df['lat'], c='red', s=100, marker='o', linewidth=0, transform=ccrs.Geodetic())

#value labels inside symbols
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', x=0, y=0)
for value, x, y in zip(df['ID'], df['lon'], df['lat']):
    plt.text(x, y, value, size=3, color='white', weight='bold', verticalalignment='center', horizontalalignment='center', transform=text_transform)

#title
plt.title('White Island Leveling Sites, siteID Numbers')

#save
plt.savefig('leveling_sites.png')